## 1. 필수 라이브러리 설치
##### LangChain과 벡터 DB Chroma, 임베딩 모델을 위한 라이브러리를 설치

##### pip install langchain langchain-community langchain-huggingface chromadb sentence-transformers

## 2. RAG 구현 코드
##### 특정 텍스트 문서(예: 상식 데이터)를 기반으로 TinyLlama가 답변

In [ ]:
import torch
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFacePipeline
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

from config import config

# 1. 모델 및 임베딩 설정 TinyLlama 모델 로드 (Fine-tuning한 모델 경로)  # # TODO !!!!!!!
model_id = config.save_path
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto")

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=256, temperature=0.1)
llm = HuggingFacePipeline(pipeline=pipe)
# # TODO HuggingFace에서 한국어 성능 좋은 임베딩 모델 찾기
embeddings = HuggingFaceEmbeddings(model_name=config.embedding_path)

# 2. 문서 및 벡터DB 설정  TODO 참조할 파일 load 하는 방식으로 수정
documents = ["사과는 비타민 C가 풍부합니다.", "대한민국의 수도는 서울입니다."]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
texts = text_splitter.create_documents(documents)
vectorstore = Chroma.from_documents(documents=texts, embedding=embeddings)
retriever = vectorstore.as_retriever()

# 3. [중요] LCEL 방식의 Chain 구성 (RetrievalQA 대체)
template = """참고 문맥을 바탕으로 질문에 답하세요:
{context}

질문: {question}
답변:"""

prompt = ChatPromptTemplate.from_template(template)

# LCEL 파이프라인 구성: 질문 -> 컨텍스트 검색 -> 프롬프트 생성 -> 모델 실행 -> 결과 출력
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 4. 실행
response = rag_chain.invoke("대한민국의 수도는 어디야?")
print(response)

## 4. 최적화 포인트
##### 1) Embedding Model (검색의 핵심)
###### HuggingFace에서 한국어 성능 좋은 임베딩 모델 찾기 (e.g ko-sroberta-multitask)

##### 2) Prompt Engineering (소형 모델의 가이드)
###### TinyLlama는 매개변수가 적기 때문에 프롬프트가 매우 중요 ### 참고 문맥:과 같이 명확한 구분자를 주어 모델이 어디를 읽고 어디에 대답해야 하는지 명시.

##### 3) Temperature 설정
###### RAG에서는 모델의 창의성보다 문서에 기반한 정확성이 중요. 따라서 temperature를 0.1에 가깝게 낮게 설정하여 멋대로 말을 지어내는 '할루시네이션'을 방지.

##### 4) 소형 모델의 한계 극복
###### TinyLlama가 문맥이 너무 길면 답변 어려움. chunk_size를 200~300 정도로 짧게 유지, k값(가져올 문서 개수)을 2~3개 정도로 제한(for 2080 Ti 메모리와 모델 성능 모두에 유리)